In [1]:
!pip install langchain openai wikipedia tiktoken neo4j

In [2]:
import os

from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.document_loaders import WikipediaLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter

os.environ['OPENAI_API_KEY'] = "API_KEY"

In [3]:
raw_documents = WikipediaLoader(query="Leonhard Euler").load()
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=20
)
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 1047, which is longer than the specified 1000
Created a chunk of size 1221, which is longer than the specified 1000


In [4]:
# Remove summary from metadata
for d in documents:
    del d.metadata['summary']

In [5]:
# Neo4j Aura credentials
url="neo4j+s://.databases.neo4j.io"
username="neo4j"
password="<insert password>"

db = Neo4jVector.from_documents(
    documents,
    OpenAIEmbeddings(),
    url=url,
    username=username,
    password=password
)

In [6]:
query = "Where did Euler grow up?"

results = db.similarity_search(query, k=1)
print(results[0].page_content)

== Early life ==
Leonhard Euler was born on 15 April 1707, in Basel, Switzerland, to Paul III Euler, a pastor of the Reformed Church, and Marguerite (née Brucker), whose ancestors include a number of well-known scholars in the classics. He was the oldest of four children, having two younger sisters, Anna Maria and Maria Magdalena, and a younger brother, Johann Heinrich. Soon after the birth of Leonhard, the Euler family moved from Basel to the town of Riehen, Switzerland, where his father became pastor in the local church and Leonhard spent most of his childhood.From a young age, Euler received schooling in mathematics from his father


In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain

chain = RetrievalQAWithSourcesChain.from_chain_type(
    ChatOpenAI(temperature=0), chain_type="stuff", retriever=db.as_retriever()
)

In [8]:
query = "What is Euler credited for popularizing?"

chain(
    {"question": query},
    return_only_outputs=True,
)

{'answer': "Euler is credited for popularizing mathematical notation and terminology, including the use of the letter e for the base of the natural logarithm, the Greek letter π to denote the ratio of a circle's circumference to its diameter, and the notation i to denote the imaginary unit. He also introduced the modern notation for functions and trigonometric functions. Additionally, Euler made important contributions to complex analysis and introduced scientific notation. He is widely recognized for his discoveries and contributions in various branches of mathematics. \n",
 'sources': 'https://en.wikipedia.org/wiki/Contributions_of_Leonhard_Euler_to_mathematics, https://en.wikipedia.org/wiki/Leonhard_Euler'}

In [9]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa = ConversationalRetrievalChain.from_llm(
    ChatOpenAI(temperature=0), db.as_retriever(), memory=memory)

In [10]:
print(qa({"question": "What is Euler credited for popularizing?"})["answer"])

Euler is credited for popularizing several mathematical notations and concepts. Some of these include:

1. The use of the Greek letter π (pi) to represent the ratio of a circle's circumference to its diameter.
2. The notation f(x) to represent the value of a function.
3. The use of the letter i to represent the imaginary unit (√-1).
4. The use of the Greek letter Σ (sigma) to represent summations.
5. The use of lowercase letters to represent the sides of a triangle and capital letters to represent the angles.
6. The definition of the constant e, now known as Euler's number, which is the base of the natural logarithm.

These contributions by Euler have had a lasting impact on the field of mathematics and are still widely used today.


In [11]:
print(qa({"question": "Where did he grow up?"})["answer"])

Euler grew up in the town of Riehen, Switzerland.
